***
*Project:* Deep Active Inference

*Author:* Jingwei Liu (Music department, UC San Diego)
***

# <span style="background-color:darkorange; color:white; padding:2px 6px">Tutorial</span> 


# Version 1: DJ - Audience Model (Experiment before choosing action)

* Actively choose my action for desired results

* Adapt my mind to the true observation

* Experiment on the environment without cost

* Minimize under categorical cross-entropy

In [1]:
import numpy as np
import pandas as pd
import csv
from scamp import *
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Flatten,Softmax,Input
import keras.backend as K 

Using TensorFlow backend.


In [2]:
df = pd.read_csv('Species.csv', sep=",",index_col = 0)
n = df.shape[0]
X = tf.Variable(np.zeros((2,n*2)),dtype='int32')  # any data tensor
for i in range(0,2*n,2):
    X[0,i].assign(df['Cantus'][i/2] - 43)
    X[1,i].assign(0)
    X[0,i+1].assign(df['Counter'][i/2] - 55)
    X[1,i+1].assign(1)
X

<tf.Variable 'Variable:0' shape=(2, 20000) dtype=int32, numpy=
array([[15, 12, 23, ..., 10, 13,  9],
       [ 0,  1,  0, ...,  1,  0,  1]])>

In [3]:
depth = 30
X = tf.concat([tf.one_hot(X[0,:], depth),tf.dtypes.cast(tf.reshape(X[1,:],[20000,1]),tf.float32)],1)
X

<tf.Tensor: shape=(20000, 31), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)>

In [4]:
train_dataset = tf.data.Dataset.from_tensor_slices(X[:18000,:])
test_dataset = tf.data.Dataset.from_tensor_slices(X[18000:,:])

In [5]:
batch_size = 18

In [6]:
n_steps = 20
window_length = n_steps
dataset = test_dataset.window(window_length, shift=2, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
shuffle_dataset = dataset.shuffle(20000).batch(batch_size,drop_remainder=True)

In [7]:
def creat_model(batch_size,Type):
    
    rnn_units = 12
    n_notes = 30

    input_A = keras.layers.Input(shape=(None,31), name="input_A") # (15, 20, 31)
    input_B = keras.layers.Input(shape=(1,31), name="input_B") # (15, 1, 31)

    hidden1 = keras.layers.LSTM(rnn_units, return_sequences=True)(input_A) # (15,20,12)
    hidden2 = keras.layers.LSTM(rnn_units, return_sequences=True)(hidden1) #(15,20,12)
    
    output_RNN = keras.layers.Dense(n_notes, activation='softmax', name = 'output_RNN')(hidden2) #(15,20,30)

    e = keras.layers.Dense(1, activation='tanh')(hidden2) #(15,20,1)
    e = keras.layers.Reshape([-1])(e) #(15,20)

    alpha = keras.layers.Activation('softmax')(e) #(15,20)
    c = keras.layers.Permute([2, 1])(keras.layers.RepeatVector(rnn_units)(alpha)) #(15,20,12)
    c = keras.layers.Multiply()([hidden2, c]) #(15,20,12)
    c = keras.layers.Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c) #(15,12)

    output_A = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_A')(c) #(15,30) (0, cantus)
    reshape = tf.reshape(output_A,[-1,1,30]) # (15, 1, 30)
    if Type == "_0_1":
        zero = tf.reshape(tf.zeros(batch_size),[batch_size,1,1])
        input_output_A = tf.concat((reshape,zero),2) # (15, 1, 31)  (0, cantus)
    elif Type == "_1_0":
        one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
        input_output_A = tf.concat((reshape,one),2) # (15, 1, 31)  (0, cantus)

    aux1 = keras.layers.SimpleRNN(rnn_units)(input_B, initial_state=c) #(15,12)
    aux2 = keras.layers.SimpleRNN(rnn_units)(input_output_A, initial_state=c) #(15,12)
    output_B1 = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_B1')(aux1) #(15,30) (1, counter)
    output_B2 = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_B2')(aux2) #(15,30) (1, counter)

    model = keras.models.Model([input_A, input_B], [output_RNN, output_A, output_B1, output_B2])
    
    aux_model1 = keras.models.Model([input_A, input_B], output_B1)
    aux_model2 = keras.models.Model(input_A, [output_A, output_B2])
    
    sub_model1 = keras.models.Model(input_A, output_A)
    sub_model2 = keras.models.Model(input_A, output_B2)
    
    att_model = keras.models.Model([input_A, input_B], alpha)
    
    return model,aux_model1,aux_model2,sub_model1,sub_model2,att_model

In [8]:
model_0_1_,aux_model1_0_1_,aux_model2_0_1_, sub_model1_0_1_,sub_model2_0_1_,att_model_0_1_ = creat_model(batch_size,'_0_1')
model_1_0_,aux_model1_1_0_,aux_model2_1_0_, sub_model1_1_0_,sub_model2_1_0_,att_model_1_0_ = creat_model(batch_size,'_1_0')

In [9]:
model_1_0_.load_weights('model_1_0_weight.h5')

In [49]:
model_0_1_.load_weights('version1_model_0_1_step1_train1.h5')

In [51]:
model_0_1_.load_weights('version1_model_0_1_step2_train1.h5')

In [53]:
model_0_1_.load_weights('version1_model_0_1_step5_train1.h5')

In [60]:
model_0_1_.load_weights('version1_model_0_1_step1_train1_random_half.h5')

In [57]:
model_0_1_.load_weights('version1_model_0_1_step2_train1_random_half.h5')

In [88]:
model_0_1_.load_weights('version1_model_0_1_step5_train1_random_half.h5')

In [81]:
model_0_1_.load_weights('version1_model_0_1_step1_train1_random.h5')

In [84]:
model_0_1_.load_weights('version1_model_0_1_step2_train1_random.h5')

In [98]:
model_0_1_.load_weights('version1_model_0_1_step5_train1_random.h5')

In [99]:
# version 1/2 generate sequence

seq_length = 10

mode = "max"
#mode = "random"

for window in shuffle_dataset.take(1):
    sequence = window
    initial = window
    
    X0 = np.zeros((batch_size,10,31))
    X1 = np.zeros((batch_size,10,31))
    
    action = np.zeros((batch_size,seq_length))
    real_observation = np.zeros((batch_size,seq_length))
    obv_weight = np.zeros((batch_size,seq_length))
    
    for s in range(seq_length):
        for g in range(10):
            X0[:,g,:] = initial[:,2*g,:]
            X1[:,g,:] = initial[:,2*g+1,:]

        [pred_o,a] = aux_model2_0_1_.predict(X0) #(18,30)
        
        for i in range(batch_size):
            if mode == "max":
                action[i,s] = np.where(a[i,:] == np.max(a[i,:]))[0][0]
            elif mode == "random":
                action[i,s] = np.random.choice(30, 1, p=a[i,:])[0]

        
        one_hot = tf.one_hot(action[:,s], 30) #(18,30)

        reshape = tf.reshape(one_hot,[batch_size,1,30])
        one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
        input_action = tf.concat((reshape,one),2) # (18, 1, 31)  (1, counter)

        real_o = aux_model1_1_0_.predict((X1, input_action)) # my action on environment; (18,30) real observation

        
        for i in range(batch_size):
            if mode == "max":
                real_observation[i,s] = np.where(real_o[i,:] == np.max(real_o[i,:]))[0][0]
            elif mode == "random":
                real_observation[i,s] = np.random.choice(30, 1, p=real_o[i,:])[0]
            
            index = int(real_observation[i,s])
            obv_weight[i,s] = pred_o[i,index]

        one_hot = tf.one_hot(real_observation[:,s], 30) #(18,30)

        reshape = tf.reshape(one_hot,[batch_size,1,30])
        zero = tf.reshape(tf.zeros(batch_size),[batch_size,1,1])
        input_observation = tf.concat((reshape,zero),2) # (18, 1, 31)  (1, counter)
        
        sequence = tf.concat((sequence,input_observation,input_action),1)
        
        initial = sequence[:,-20:,:]
        
  
    print('action:',action)
    print('observation:',real_observation)
    print('')
    
    
    step = [];
    for j in range(seq_length):
        step.append("step" + str(j))
    step.append("sequence match")
        
    sample = [];
    for i in range(batch_size):
        sample.append("sequence" + str(i))
    sample.append("step match")
    
    table = [];
    for i in range(batch_size):
        seq = [];
        for j in range(seq_length):
            seq.append(str(int(action[i,j])) + "/" + str(int(real_observation[i,j])) + "/(" + '%.2f'%(obv_weight[i,j]) + ")")
        match = np.sum(obv_weight[i,:])/seq_length
        seq.append('%.2f'%(match))
        table.append(seq)
        
    last_row = [];
    for j in range(seq_length):
        match = np.sum(obv_weight[:,j])/batch_size
        last_row.append('%.2f'%(match))
    total = np.sum(obv_weight)/(seq_length*batch_size)
    last_row.append('%.2f'%(total))
    table.append(last_row)
    
    format_row = "{:>12}" * (seq_length + 2)
    print(format_row.format("act/obv/(pred_obv_weight)", *step))
    for team, row in zip(sample, table):
        print(format_row.format(team, *row))

action: [[25. 25. 25. 25. 25. 25. 22. 22. 22. 22.]
 [ 1.  1.  1.  1.  6.  9.  9.  9. 25. 25.]
 [22. 22. 22. 22. 22. 22. 22. 21. 22. 22.]
 [22. 22. 20. 20. 20. 20. 25. 25. 25. 24.]
 [25. 25. 25. 25. 25. 22. 22. 22. 22. 22.]
 [25. 25. 25. 25. 25. 22. 22. 22. 22. 22.]
 [ 6.  6.  6.  9. 25. 25. 25. 25. 25. 25.]
 [25. 25. 25. 25. 25. 25. 25. 25. 22. 22.]
 [ 0.  0.  0.  0.  0. 24.  0.  0.  0.  0.]
 [25. 25. 25. 25. 22. 22. 22. 22. 22. 22.]
 [ 0.  0.  0. 24. 24. 24. 24. 24. 24. 24.]
 [ 0. 25. 25. 25. 25. 25. 25. 24. 24. 24.]
 [ 0.  0.  0.  0. 24.  0.  0.  0.  0.  0.]
 [25. 25. 25. 25. 25. 25. 25. 22. 22. 22.]
 [25. 25. 25. 25. 25. 25. 25. 24. 24. 22.]
 [22. 22. 22. 22. 20. 20. 20. 22. 22. 22.]
 [24. 25. 25. 25. 25. 25. 25. 25. 24. 22.]
 [25. 25. 25. 25. 22. 22. 22. 22. 22. 22.]]
observation: [[25. 29. 29. 29. 29. 29. 25. 26. 26. 26.]
 [24. 10. 10. 10. 14. 18. 18. 18. 29. 29.]
 [26. 26. 26. 26. 26. 26. 26. 29. 26. 26.]
 [22. 22. 24. 24. 24. 24. 29. 29. 29. 28.]
 [29. 29. 29. 29. 29. 22. 22. 26

In [19]:
sample = 3
music = tf.reshape(sequence[6,:,:],[-1,31])
#music = music[-20:,:]
row,col = music.shape
cantus_lst = [];
counter_lst = [];
for i in range(row):
    clas = np.where(music[i,:-1] == 1)[0][0]
    if music[i,-1] == 0:
        cantus_lst.append(clas+43)
    elif music[i,-1] == 1:
        counter_lst.append(clas+55)

In [24]:
cantus_lst

[70, 66, 71, 69, 65, 62, 58, 51, 52, 49, 52, 52, 52, 52, 52, 52, 52, 51, 51]

In [25]:
counter_lst

[77, 75, 74, 73, 68, 70, 62, 63, 59, 57, 55, 55, 55, 55, 55, 55, 55, 55, 55]

In [26]:
np.array(counter_lst) - np.array(cantus_lst) # real harmonic

array([ 7,  9,  3,  4,  3,  8,  4, 12,  7,  8,  3,  3,  3,  3,  3,  3,  3,
        4,  4], dtype=int64)

In [27]:
s = Session()
s.tempo = 180
piano1 = s.new_part("piano")
piano2 = s.new_part("piano")

def cantus():
    for i in cantus_lst:
        piano2.play_note(i,1,4)
        
def counter():
    for i in counter_lst:
        piano1.play_note(i,1,4)
        
        
s.fast_forward_to_beat(100)

s.start_transcribing()
s.fork(counter)
s.fork(cantus)
s.wait_for_children_to_finish()
performance = s.stop_transcribing()
performance.to_score(title = "First Species Counterpoint", composer = "My programme",time_signature = "4/4").show_xml()

Using preset Piano Merlin for piano
Using preset Piano Merlin for piano
